# 第12章 类和动态内存分配

## 12.1 动态内存和类

### 12.1.1 复习示例和静态类成员

#### [程序清单 12.1 stringbad.h](notebook_codes/code_12_1_stringbad.h)

#### [程序清单 12.2 stringbad.cpp](notebook_codes/code_12_2_stringbad.cpp)

**注意：**静态数据成员在类声明中声明，在包含类方法的文件中初始化。初始化时使用作用域运算符来指出静态成员所属的类。但如果静态成员时整型或者枚举型const，则可以在类声明中初始化。

#### [程序清单 12.3 vegnews.cpp](notebook_codes/code_12_3_vegnews.cpp)


### 12.1.2 特殊成员函数
C++ 自动提供了下面这些成员函数：
- 默认构造函数，如果没有定义构造函数
- 默认析构函数，如果没有定义
- 复制构造函数，如果没有定义
- 赋值运算符，如果没有定义
- 地址运算符，如果没有定义

#### 1. 默认构造函数
如果没有提供任何构造函数，C++将创建默认构造函数。例如，假如定义了一个`Klunk`类，但没有提供任何构造函数，则编译器将提供下述默认构造函数：
```cpp
Klunk::Klunk() { } // implicit default constructor
```

带参数的构造函数也可以是默认构造函数，只要所有参数都有默认值。例如，Klunk类可以包含下述内联构造函数：
```cpp
Klunk(int n = 0) { klunk_ct = n; }
```

但是只能有一个默认构造函数，也就是说，不能这么做：
```cpp
Klunk() { klunk_ct = n; }           // constructor #1
Klunk(int n = 0) { klunk_ct = n; }  // ambiguous constructor #2
```

#### 2. 复制构造函数
复制构造函数用于将一个对象复制到新创建的对象中。也就是说，它用于初始化过程中(包括按值传递参数)，而不是常规的赋值过程中，类的复制构造函数原型如下：
```cpp
Class_name(const Class_name &);
```

它接受一个指向类对象的常量引用作为参数，例如，`String`类的复制构造函数的原型如下：
```cpp
StringBad(const StringBad &);
```

#### 3. 何时调用复制构造函数

新建一个对象，将其初始化为同类现有对象时，复制构造函数都将被调用。最常见的情况是将新对象显式地初始化为现有的对象。例如以下四种声明：
```cpp
StringBad ditto(motto);
StringBad metoo = motto;
StringBAd also = StringBad(motto);
StringBad * pStringBad = new StringBad(motto);
```

当函数按值传递对象时(例如程序清单12.3中的callme2())或函数返回对象时，都将使用复制构造函数。记住，按值传递意味着创建原始变量的一个副本。

#### 4. 默认的复制构造函数的功能

默认的复制构造函数逐个复制非静态成员(成员复制也称为浅复制)，复制的是成员的值。


### 12.1.3 回到Stringbad: 复制构造函数哪里出了问题

1. 析构函数的调用次数比构造函数调用次数多：因为在调用callme2()函数时，复制构造函数被用来初始化callme2()的形参
2. 字符串内容乱码：因为隐式复制构造函数是按值进行复制的，复制了一个指向字符串的指针而不是字符串本身。因此当析构函数被调用时，会导致多次释放内存。

#### 1. 定义一个显式复制构造函数来解决问题

解决类设计中这种问题的方法是进行深度复制(deep copy)。也就是说，复制构造函数应当复制字符串并将副本的地址赋给str成员，而不仅仅是复制字符串地址。

```cpp
StringBad::StringBad(const StringBad & st) {
    num_strings++;
    len = st.len;
    str = new char [len+1];
    std::strcpy(str, st.str);
    cout << num_strings << ": \"" << str
         << "\" object created\n";
}
```

**警告：**如果类中包含了使用new初始化的指针成员，应当定义一个复制构造函数，以复制指向的数据，而不是指针，这被称为深度复制。复制的另一种形式(成员复制或浅复制)只是复制指针值。

### 12.1.4 Stringbad的其他问题：赋值运算符

赋值运算符的原型如下：
```cpp
Class_name & Class_name::operator=(const Class_name &);
```

#### 1. 赋值运算符的功能以及何时使用它
将已有的对象赋给另一个对象时，将使用重载的赋值运算符。
```cpp
StringBad headline1("Celery Stalks at Midnight");
StringBad knot;
knot = headline1;  // assignemnt operator invoked
```

初始化对象时，并不一定会使用赋值运算符：
```cpp
StringBad metto = knot; // use copy constructor, possibly assignment too
```

#### 2. 赋值的问题出在哪里

出现的问题与隐式复制构造函数相同，试图删除前面已经删除的字符串。

#### 3. 解决赋值的问题

解决办法是提供赋值运算符(进行深度赋值)定义。其实现与复制构造函数相似，但也有一些差别。

- 由于目标对象可能引用了以前分配的数据，所以函数应使用 delete [] 来释放这些数据。
- 函数应当避免将对象赋给自身：否则，给对象重新赋值前，释放内存操作可能删除对象的内存。
- 函数返回一个指向对象的引用。

```cpp
StringBad & StringBad::operator=(const StringBad & st) {
    if (this == &st)
        return *this;
    delete [] str;
    len = st.len;
    str = new char [len + 1];
    std::strcpy(str, st.str);
    return *this;
}
```

---

## 12.2 改进后的新String类

### 12.2.1 修订后的默认构造函数

```cpp
String::String() {
    len = 0;
    str = new char[1];
    str[0] = '\0';        // default string
}
```

#### C++11空指针
C++98中，字面值0有两个含义：
- 数值零
- 空指针

这使得阅读程序的人和编译器难以区分。有些程序员使用`(void *) 0`来标识空指针，还有的程序员使用NULL，这是一个标识空指针的C语言宏。

C++11提供了更好的解决方案：引入新关键字`nullptr`，用于标识空指针。您仍然可以像以前一样使用0，但更建议使用`nullptr`


### 12.2.2 比较成员函数

```cpp
bool operator<(const String &st1, const String &st2) {
    return (std::strcmp(st.str, st2.str) < 0);
}

bool operator>(const String &st1, const String &st2) {
    return st2 < st1;
}

bool operator==(const String &st1, const String &st2) {
    return (std::strcmp(st1.str, st2.str) == 0);
}
```


### 12.2.3 使用中括号表示法访问字符

C++中，两个中括号组成一个运算符--中括号运算符，可以使用方法`operator[]()`来重载该运算符。对于中括号运算符(二元运算符)，第一个操作数位于中括号前面，另一个操作数位于两个中括号之间。因此，在表达式`city[0]`中，city是第一个操作数，`[]`是运算符，0是第二个操作数。

```cpp
char & String::operator[](int i) const {
    return str[i];
}
```

### 12.2.4 静态类成员函数

可以将成员函数声明成静态的(函数声明必须包含关键字`static`，但如果函数定义是独立的，则其中不能包含关键字`static`)，这样做有两个重要的后果：
- 不能通过对象调用静态成员函数；实际上，静态成员函数甚至不能使用`this`指针。如果静态成员函数是在公有部分声明的，则可以使用类名和作用域解析运算符来调用它。例如，可以给`String`类添加一个名为`How Many()`的静态成员函数，方法是在类声明中添加如下原型/定义：
```cpp
static int HowMany() { return num_strings; }
```

调用方式如下：
```cpp
int count = String::HowMany();  // invoking a static member function
```

- 第二点，由于静态成员函数不与特定的对象相关联，因此只能使用静态数据成员。

### 12.2.5 进一步重载赋值运算符

当我们想要把一个用`getline()`读取的字符串，赋值给`String`对象时，步骤如下：
1. 使用构造函数`String(const char *)`创建一个临时`String`对象
2. 再使用`String::operator=(const String &)`函数创建临时对象中的信息复制过去。
3. 程序调用`~String()`析构函数删除临时对象。

可以用重载赋值运算符来提高效率：
```cpp
String & String::operator=(const char * s) {
    delete [] str;
    len = std::strlen(s);
    str = new char[len + 1];
    str::strcpy(str, s);
    return *this;
}
```

---

## 12.3 在构造函数中使用new时应该注意的事项

在使用`new`初始化对象的指针成员时，需要注意以下几点：
1. 如果在构造函数中使用`new`来初始化指针成员，则应在析构函数中使用`delete`
2. `new` 和 `delete` 必须相互兼容，`new`对应与`delete`，`new[]`对应与`delete[]`。
3. 如果有多个构造函数，则必须以相同的方式使用`new`，要么都带中括号，要么都不带。因为只有一个析构函数，所有的构造函数都必须与它兼容。然而，可以在一个构造函数中使用`new`初始化指针，而在另一个构造函数中将指针初始化为空(0或C++11中的nullptr)，这是因为delete(不论是带中括号还是不带中括号)可以用于空指针。
4. 应定义一个复制构造函数，通过深度复制将一个对象初始化为另一个对象。通常，这种构造函数与下面类似。
```cpp
String::String(const String & st) {
    num_strings++;
    len = st.len;
    str = new char[len+1];
    std::strcpy(str, st.str);
}
```
5. 应当定义一个赋值运算符，通过深度复制将一个对象复制给另一个对象。通常，该类方法与下面类似：
```cpp
String & String::operator=(const String & st) {
    if (this == &st)
        return *this;
    delete [] str;
    len = st.len;
    str = new char[len+1];
    std::strcpy(str, st.str);
    return *this;
}
```


### 12.3.1 应该和不应该

### 12.3.2 包含类成员的类的逐成员复制

---

## 12.4 有关返回对象的说明

### 12.4.1 返回指向const对象的引用

### 12.4.2 返回指向非const对象的引用

### 12.4.3 返回对象

### 12.4.4 返回const 对象

---

## 12.5 使用指向对象的指针

### 12.5.1 再谈new和delete

### 12.5.2 指针和对象小结

### 12.5.3 再谈定位new运算符

---

## 12.6 复习各种技术

### 12.6.1 重载<<预算符

### 12.6.2 转换函数

### 12.6.3 其构造函数使用new的类

---

## 12.7 队列模拟

### 12.7.1 队列类

#### 成员初始化列表的语法

如果Classy是一个类，而`mem1`、`mem2`和`mem3`都是这个类的数据成员，则类构造函数可以使用如下的语法来初始化数据成员：
```cpp
Classy::Classy(int n, int m) : mem1(n), mem2(0), mem3(n*m + 2) {
    ...
}
```

### 12.7.2 Customer 类

### 12.7.3 ATM 模拟

---